# Co-training

1. We assume only part of label exist

2. We only select binary case (Only when one name indicate two and only two author)

3. When we apply 10 fold with co-training, each fold of first iteration will be baseline compare to co-training

In [1]:
import os
import sys
import warnings

#warnings.filterwarnings('error')
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
np.set_printoptions(precision=4, suppress=True)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import com_func

#----- threshold for selecting set of name group -----------#
threshold_select_name_group = 100
#----- threshold for selecting min sample in name group ----#
threshold_lower = 100
threshold_upper = 110

apply_threshold_to_name_group_samples = True

# pp_text = ["lsa", "pv_dm", "pv_dbow"]
pp_text = ["pv_dbow"]
pp_citation = "n2v"

Dataset = "pubmed"

In [2]:
import numpy as np
import warnings
import matplotlib.pyplot as plt

from collections import defaultdict

# create co training classifier
class Co_training_clf(object):
    
    import copy
    
    def __init__(self, clf1, clf2=None, p=1, n=1, k=30, u = 75):
        
        self.clf1 = clf1
        # assume co_training on one classifier
        if clf2 == None:
            self.clf2 = self.copy.deepcopy(clf1)
        else:
            self.clf2 = clf2
        # take p example from most confidently positive labels to example
        self.p = p
        # take n example from most confidently negative label to example
        self.n = n
        # number of iteration
        self.k = k
        # size of pool of unlabeled samples
        self.u = u

    def init_L_U_U_prime(self, labels):
        # index of the samples that are initially labeled
        L = [i for i, label_i in enumerate(labels) if label_i != -1]
        # index of unlabeled samples
        U = [i for i, label_i in enumerate(labels) if label_i == -1]
        print("Initial L size: ", len(L))
        print("Initial U size: ", len(U))
        # random drawing sample from U
        random.shuffle(U)
        U_prime = U[-min(len(U), self.u):]
        # remove the samples in U_prime from U
        U = U[:-len(U_prime)]
        return L, U, U_prime
    
    def check_iter_label_mapping(self, iter_clf1, iter_clf2):
        '''
        In theory, it shouldn't occur that label not mapping since it trained on same dataset but different view
        But add a check to make sure it won't occur and save the class mapping for late label unlabeled sample
        '''
        dv1_class_label = iter_clf1.classes_
        dv2_class_label = iter_clf2.classes_
        if all(dv1_class_label == dv2_class_label):
            self.class_ = dv1_class_label
        else:
            sys.exit("Two view classifier label not mapping")

    def label_p_n_samples(self, proba, rank):
        U_prime_size = len(proba)
        self_trained_labels = []
        for label, conf_measure in enumerate(rank):
            # 0 positive sample
            if label==0:
                p = []
                index = 0
                while(len(p) < self.p):
                    max_conf_sample_index = conf_measure[index]
                    # ---- if positive predict proba is more than 50% ------- #
                    if (proba[max_conf_sample_index][label] > 0.5):
                        print('P: ', max_conf_sample_index, " : ", proba[max_conf_sample_index])
                        p.append(max_conf_sample_index)
                    index +=1
                    if (index>=U_prime_size):
                        break
                self_trained_labels.append(p)
            # 1 negative sample
            elif label == 1:
                n = []
                index = 0
                while(len(n) < self.n):
                    max_conf_sample_index = conf_measure[index]
                    # ---- if negative predict proba is more than 50% ------- #
                    if (proba[max_conf_sample_index][label] > 0.5):
                        print('N: ', max_conf_sample_index, " : ", proba[max_conf_sample_index])
                        n.append(max_conf_sample_index)
                    index +=1
                    if (index>=U_prime_size):
                        break
                self_trained_labels.append(n)
            else:
                print("Class label error")
        return self_trained_labels

    def get_self_labeled_sample(self):
        '''
        return:
            self-labeled new positive, self-labeled new negative (Index)
        '''
        
        return self.new_labeled_idx

    def plot_co_training_process(self, iterCount, data, iter_train_label, unlabeled_idx, new_sample_idx, plotSavingPath, name):
        if not os.path.exists(plotSavingPath):
            os.makedirs(plotSavingPath)
        # apply PCA on input data
        pca = PCA(n_components=2)
        pca_transformed = pca.fit_transform(X=data)
        pca_one = pca_transformed[:,0]
        pca_two = pca_transformed[:,1]
        # Layer 1. plot the labeled samples
        fig, ax = plt.subplots(figsize=(9,7))
        for author in np.unique(iter_train_label):
            ix = iter_train_label.index[iter_train_label == author].tolist()
            # print(ix)
            ax.scatter(pca_one[ix], pca_two[ix], cmap='viridis', label = author, s = 50, alpha = 0.5)
        # Layer 2, plot unlabel samples in u_prime
        ax.scatter(pca_one[unlabeled_idx], pca_two[unlabeled_idx], color='grey', label = "unlabeled", s = 50, alpha = 0.5)
        # layer 3, mark self labeled result
        temp = ax.scatter(pca_one[new_sample_idx], pca_two[new_sample_idx], edgecolor='black', linewidth='1', s=50)
        temp.set_facecolor("none")
        temp.set_label("self-labeled")
        legend = ax.legend(loc='upper left', bbox_to_anchor=(0, 1.2), ncol=2)
        plt.title('Co-training iteration: '+ str(iterCount), fontsize=14)
        plt.xlabel("PCA one",fontsize=14)
        plt.ylabel("PCA two",fontsize=14)
        plt.savefig((plotSavingPath+name+"_PCA_i-"+str(iterCount)+".png").encode('utf-8'), dpi=100, bbox_extra_artists=(legend,), bbox_inches='tight')
        plt.close("all")
        # plt.show()
        

    def fit(self, dataView1, dataView2, labels, dv1_test, dv2_test, label_test, plot_save_name=None, plot_save_path=None):
        # index of self labeled samples
        self.new_labeled_idx = defaultdict(list)
        # when fit co-train, we collect f1 on test samples wrt each iteration
        self.f1_on_test_dv1 = []
        self.f1_on_test_dv2 = []
        
        print("P value: ", self.p, " N value: ", self.n)
        
        L, U, U_prime = self.init_L_U_U_prime(labels)
        # print("L: ", L)
        
        train_idx = dataView1.index.values
        labels = pd.Series(labels, index = train_idx)
        iterCount = 0
        # --------- plot initial stage -------------- #
        init_train_label = labels[L]
        if plot_save_path != None:
            plot_save_dv1_name = plot_save_name+"_dv1"
            # ----- save pca reduced plot for dv1 ------ #
            self.plot_co_training_process(iterCount, dataView1, init_train_label, U_prime, [],
                                          plot_save_path, plot_save_dv1_name)
            # ----- dv2 -------- #
            plot_save_dv2_name = plot_save_name+"_dv2"
            self.plot_co_training_process(iterCount, dataView2, init_train_label, U_prime, [],
                                          plot_save_path, plot_save_dv2_name)
        
        #loop until we have assigned labels to every sample in U and U_prime or we hit our iteration break condition
        while iterCount < self.k and U_prime:
            # print("step",iterCount, " L: ",L)
            # print("step",iterCount, " U_prime: ",U_prime)
            # ------------- get labeled samples for train ----------- # 
            iter_train_d1 = dataView1.iloc[L]
            iter_train_d2 = dataView2.iloc[L]
            iter_train_label = labels[L]
            # print(iter_train_label)
            # ----------- get U_prime unlabeled samples  ------------ #
            iter_unlabeled_d1 = dataView1.iloc[U_prime]
            iter_unlabeled_d2 = dataView2.iloc[U_prime]
            # ------------ train different view classifier ----------- #
            iter_clf1 = self.copy.deepcopy(self.clf1) 
            iter_clf2 = self.copy.deepcopy(self.clf2)
            iter_clf1.fit(iter_train_d1, iter_train_label.ravel())
            iter_clf2.fit(iter_train_d2, iter_train_label.ravel())
            self.check_iter_label_mapping(iter_clf1, iter_clf2)
            # --------- test error on test data --------------------- #
            # make prediction on test data
            y1 = iter_clf1.predict(dv1_test)
            y2 = iter_clf2.predict(dv2_test)
            # f1 score on each iteration
            f1_dv1 = f1_score(label_test, y1, average='macro')
            f1_dv2 = f1_score(label_test, y2, average='macro')
            # collect f1 for current iteration
            self.f1_on_test_dv1.append(f1_dv1)
            self.f1_on_test_dv2.append(f1_dv2)
            # rank class probabilities for unlabeled sample for it's confidence measure
            dv1_proba = iter_clf1.predict_proba(iter_unlabeled_d1)
            dv2_proba = iter_clf2.predict_proba(iter_unlabeled_d2)
            dv1_proba_rank = []
            dv2_proba_rank = []
            # proba1_rank[i] is label i's confidence measure
            for class_proba in dv1_proba.T:
                dv1_proba_rank.append((-class_proba).argsort())
            for class_proba in dv2_proba.T:
                dv2_proba_rank.append((-class_proba).argsort())
            # print(dv1_proba)
            # print(dv1_proba_rank)
            # print(dv2_proba)
            # print(dv2_proba_rank)
            # h1 classifier
            newly_labeled_dv1 = self.label_p_n_samples(dv1_proba, dv1_proba_rank)
            # h2 classifier
            newly_labeled_dv2 = self.label_p_n_samples(dv2_proba, dv2_proba_rank)
            roundNew = list(zip(newly_labeled_dv1, newly_labeled_dv2))
            # auto label the samples and remove it from U_prime
            round_auto_labeled = []
            for label, round_new in enumerate(roundNew):
                round_new = set([item for sublist in round_new for item in sublist])
                auto_labeled = [U_prime[x] for x in round_new]
                round_auto_labeled.extend(auto_labeled)
                self.new_labeled_idx[self.class_[label]].append(auto_labeled)
                # add label to those new samples
                labels[auto_labeled] = self.class_[label]
                print(self.class_[label]," (u' idx): ",round_new)
                print(self.class_[label]," (U idx): ",auto_labeled)
            print(roundNew)
            print(round_auto_labeled)
            # extend the labeled sample
            L.extend(round_auto_labeled)
            # remove the labeled sample from U_prime
            U_prime = [x for x in U_prime if x not in round_auto_labeled]
            #print(U_prime)
            # randomly choice 2p+2n examples from u to replenish u_prime
            replenishItem = U[-(2*self.p+2*self.n):]
            U_prime.extend(replenishItem)
            U = U[:-len(replenishItem)]
            iterCount +=1
            # ----------- plot the co-training process -------------- #
            if plot_save_path != None:
                new_train_label = labels[L]
                self_labeled_idx_temp = [idx for idx in self.new_labeled_idx.values()]
                # print(self_labeled_idx_temp)
                last_iter_self_labeled_idx = [val for sublist in self_labeled_idx_temp for subsublist in sublist for val in subsublist]
                # print(last_iter_self_labeled_idx)
                # ----- save pca reduced plot for dv1 ------ #
                plot_save_dv1_name = plot_save_name+"_dv1"
                self.plot_co_training_process(iterCount, dataView1, new_train_label, U_prime, last_iter_self_labeled_idx,
                                              plot_save_path, plot_save_dv1_name)
                # ----- dv2 -------- #
                plot_save_dv2_name = plot_save_name+"_dv2"
                self.plot_co_training_process(iterCount, dataView2, new_train_label, U_prime, last_iter_self_labeled_idx,
                                              plot_save_path, plot_save_dv2_name)
        print("Total Labeled number: ", len(L), " Still unlabeled number: ", len(U_prime))
        print(self.k)
        print(iterCount)
        # final train
        newtrain_d1 = dataView1.iloc[L]
        newtrain_d2 = dataView2.iloc[L]
        self.clf1.fit(newtrain_d1, labels.iloc[L])
        self.clf2.fit(newtrain_d2, labels.iloc[L])
        # ------ save f1 vs number of iteration plot ------- #
        if plot_save_path != None:
            default_text_based = [self.f1_on_test_dv1[0]] * iterCount
            default_citation_based = [self.f1_on_test_dv2[0]] * iterCount
            default_step = np.arange(0,iterCount)
            co_train_text_based = self.f1_on_test_dv1[1:]
            co_train_citation_based = self.f1_on_test_dv2[1:]
            co_training_step = np.arange(1,iterCount)

            fig = plt.figure()
            ax = plt.axes()
            plt.plot(default_step, default_text_based, linestyle='dashed', label="Text based default")
            plt.plot(default_step, default_citation_based, linestyle='dashdot', label="Citation based default")
            plt.plot(co_training_step, co_train_text_based, linestyle='solid', marker = "*", label="Text based")
            plt.plot(co_training_step, co_train_citation_based, linestyle='dotted', marker = "+", label="Citation based")
            ax.autoscale_view()
            legend = ax.legend(loc='upper left', bbox_to_anchor=(0, 1.2), ncol=2)
            plt.xlabel('Co-Training Iterations')
            plt.ylabel('F1 score')
            plt.savefig((plot_save_path+plot_save_name+"_diff_iter_f1.png"), dpi=300, bbox_extra_artists=(legend,), bbox_inches='tight')
            # plt.show()
            plt.close("all")
    
    def supports_proba(self, clf, x):
        try:
            clf.predict_proba([x])
            return True
        except:
            return False

    def predict(self, dataView1, dataView2):
        y1 = self.clf1.predict(dataView1)
        y2 = self.clf2.predict(dataView2)
        proba_supported = self.supports_proba(self.clf1, dataView1.iloc[0]) and self.supports_proba(self.clf2, dataView2.iloc[0])
        #fill pred with -1 so we can identify the samples in which sample classifiers failed to agree
        y_pred = ["-1"] * dataView1.shape[0]
        for i, (y1_i, y2_i) in enumerate(zip(y1, y2)):
            # if both agree on label
            if y1_i == y2_i:
                y_pred[i] = y1_i
            # if disagree on label, times probability together, choice the class have higher probabilities
            elif proba_supported:
                y1_probas = self.clf1.predict_proba([dataView1.iloc[i]])[0]
                y2_probas = self.clf2.predict_proba([dataView2.iloc[i]])[0]
                print("y1 disagree on",i, " Proba: ",y1_probas)
                print("y2 not aggreed on ",i, "Proba: ", y2_probas)
                prod_y_probas = [proba_y1 * proba_y2 for (proba_y1, proba_y2) in zip(y1_probas, y2_probas)]
                print("product probas:",prod_y_probas)
                max_prob_idx = prod_y_probas.index(max(prod_y_probas))
                y_pred[i] = self.class_[max_prob_idx]
                print("result idx: ", max_prob_idx, " result: ",y_pred[i])
            else:
                #the classifiers disagree and don't support probability, exit
                sys.exit("classifiers disagree with label, result may not accurate")
        # convert final result to np array
        y_pred_np_array = np.asarray(y_pred)
        return y_pred_np_array

    def predict_proba(self, dataView1, dataView2):
        # the predicted probabilities is simply a product (*) of probabilities given from each classifier trained
        y1_probas = self.clf1.predict_proba(dataView1)
        y2_probas = self.clf2.predict_proba(dataView2)
        
        proba = (y1_probas*y2_probas)
        return proba


In [3]:
import copy
import random

import seaborn as sns

from collections import Counter
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score,f1_score,accuracy_score

# cross validation
def k_fold_cv_co_train_binary(dataview1, dataview2, label, init_labeled_size, clf, k=10, plot_save_name=None, plot_save_path=None):
    random.seed(1)
    kf = StratifiedKFold(n_splits=k)
    allTrueLabel = []
    allPredLabel_co_train = []
    allPredLabel_LR = []
    allPredLabel_SVM = []
    
    # obtain data ratio
    c = Counter(label)
    data_ratio = [(i, c[i] / len(label)) for i in c]
    print(data_ratio)
    
    all_fold_statistic = []
    fold = 0
    
    for train_index, test_index in kf.split(dataview1, label):
        fold +=1
        if plot_save_path !=None:
            detailed_plot_path = plot_save_path+plot_save_name+"/fold"+str(fold)+"/"
        else:
            detailed_plot_path = plot_save_path
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # ---------------split train and test -------------------- #
        dv1_train, dv1_test = dataview1.iloc[train_index], dataview1.iloc[test_index]
        dv2_train, dv2_test = dataview2.iloc[train_index], dataview2.iloc[test_index]
        all_label_train, label_test = label.iloc[train_index], label.iloc[test_index]
        # ----------- set some labeled data as unlabeled ------------ #
        # 1. use co_train_per_class_size to draw "init_labeled_size" of samples as labeled, other as unlabeled
        co_train_per_class_size = [(label, round(ratio*init_labeled_size)) for label, ratio in data_ratio]
        final_train_label = all_label_train.tolist()
        train_sample_idx = []
        # 2. mark other as unlabeled
        for unique_label, training_size in co_train_per_class_size:
            curr_label_idx = [i for i, x in enumerate(final_train_label) if x == unique_label]
            curr_label_size = len(curr_label_idx)
            unlabeled_size = curr_label_size - training_size
            unlabel_item_idx = random.sample(curr_label_idx, unlabeled_size)
            train_sample_idx += [x for x in curr_label_idx if x not in unlabel_item_idx]
            for unlabel_idx in unlabel_item_idx:
                final_train_label[unlabel_idx]=-1
        print(final_train_label)
        unlabeled_sample_size = len(final_train_label)-len(train_sample_idx)
        final_dv1 = dv1_train.reset_index(drop=True)
        final_dv2 = dv2_train.reset_index(drop=True)
        ''' -------------- train binary co-training ------------------- '''
        per_fold_clf = copy.deepcopy(clf)
        per_fold_clf.fit(final_dv1, final_dv2, final_train_label, dv1_test, dv2_test, label_test, plot_save_name, detailed_plot_path)
        # get self-labeled sample index #
        self_labeled_index = per_fold_clf.get_self_labeled_sample()
        print("Self labeled sample index: ", self_labeled_index)
        self_labeled_idx_temp = [idx for idx in self_labeled_index.values()]
        all_self_labeled_index = [val for sublist in self_labeled_idx_temp for subsublist in sublist for val in subsublist]
        # -------- use concatenated features for comparsion -------- #
        concatenated_train = pd.concat([final_dv1.iloc[train_sample_idx],final_dv2.iloc[train_sample_idx]], axis=1, ignore_index=True)
        train_label = [final_train_label[i] for i in train_sample_idx]
        ''' --- train LR on concatenated features with "init_labeled_size" labeled samples  --- '''
        LR_clf = LogisticRegression(solver= "liblinear")
        LR_clf.fit(concatenated_train, train_label)
        ''' --- train SVM on concatenated features with "init_labeled_size" labeled samples  --- '''
        SVM_clf = SVC(gamma="auto", kernel='linear')
        SVM_clf.fit(concatenated_train, train_label)
        # ------------ generate concatenated test dataset ------------ #
        concatenated_test = pd.concat([dv1_test,dv2_test], axis=1, ignore_index=True)
        # ------------- get predicted label for test set ------------- #
        co_lr_label_predict = per_fold_clf.predict(dv1_test, dv2_test)
        LR_predict = LR_clf.predict(concatenated_test)
        SVM_predict = SVM_clf.predict(concatenated_test)
        
        print("co-train f1: ", metrics.classification_report(label_test, co_lr_label_predict))
        print("LR f1: ", metrics.classification_report(label_test, LR_predict) )
        print("SVM f1: ", metrics.classification_report(label_test, SVM_predict))
        
        allTrueLabel.extend(label_test.values.tolist())
        allPredLabel_co_train.extend(co_lr_label_predict)
        allPredLabel_LR.extend(LR_predict)
        allPredLabel_SVM.extend(SVM_predict)
        # collect per fold statistic
        curr_fold_statistic = {'author': plot_save_name, 'fold':fold, 'train_size': co_train_per_class_size, 'test_size': dv1_test.shape[0],
                               'total_self_labeled_train': len(all_self_labeled_index), "unlabeled size": unlabeled_sample_size,
                               'co-train f1': f1_score(label_test.values.tolist(), co_lr_label_predict,average='macro'),
                               'LR f1': f1_score(label_test.values.tolist(), LR_predict,average='macro'),
                               'SVM f1': f1_score(label_test.values.tolist(), SVM_predict,average='macro')}
        all_fold_statistic.append(curr_fold_statistic)
        
    if plot_save_path !=None:
        # --------------- plot per fold result f1 variance --------------- #
        all_per_fold_f1_score_variance_plot = pd.DataFrame(all_fold_statistic)
        plot_temp_data = all_per_fold_f1_score_variance_plot[['co-train f1', 'LR f1', 'SVM f1']].copy()
        plot_temp_data = pd.melt(plot_temp_data, var_name='methods', value_name='f1')
        ax = sns.boxplot(x="methods", y="f1", data=plot_temp_data)
        ax = sns.swarmplot(x="methods", y="f1", data=plot_temp_data, color=".25")
        ax.set_title(plot_save_name+" result variance with 10 fold")
        plt.savefig(plot_save_path+plot_save_name+"/"+plot_save_name+"_result_variance.png", dpi=300)
        plt.show()

    co_train_accuracy = accuracy_score(allTrueLabel, allPredLabel_co_train)
    co_train_f1 = f1_score(allTrueLabel, allPredLabel_co_train,average='macro')
    print(metrics.classification_report(allTrueLabel, allPredLabel_co_train))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel_co_train).ravel())
    
    LR_accuracy = accuracy_score(allTrueLabel, allPredLabel_LR)
    LR_f1 = f1_score(allTrueLabel, allPredLabel_LR,average='macro')
    print(metrics.classification_report(allTrueLabel, allPredLabel_LR))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel_LR).ravel())
    
    SVM_accuracy = accuracy_score(allTrueLabel, allPredLabel_SVM)
    SVM_f1 = f1_score(allTrueLabel, allPredLabel_SVM,average='macro')
    print(metrics.classification_report(allTrueLabel, allPredLabel_SVM))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel_SVM).ravel())
    
    return LR_f1, SVM_f1, co_train_f1, all_fold_statistic

In [4]:
import collections
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

# fix random seed for reproducibility
np.random.seed(1)

# loop through all files in directory add name to name list
fileDir = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)

init_labeled_size = 10

co_lr_diff_embedding_result = []

#---------------- load different embeddings for view one ---------------#
for select_emb in pp_text:
    print("Load text embedding: ", select_emb)
    # read viewone embeddings
    viewone_text_emb, viewone_emb_pid = com_func.read_text_embedding(emb_type=select_emb, training_size = "140k")
    viewone_text_emb = np.column_stack((viewone_emb_pid,viewone_text_emb))
    # read viewtwo embedding, notice here we only use labeled data
    print("Load citation embedding: ", pp_citation)
    viewtwo_citation_embedding = com_func.read_citation_embedding_sorted(emb_type = pp_citation, labeled_only = True)
    # print(viewone_text_emb[0])
    # print(viewtwo_citation_embedding[0])
    
    threshold_change_all_co_lr_f1s = []
    threshold_change = []
    
    # -------------- different threshold (step by 10) -----------------------#
    for step_threshold in range(threshold_lower, threshold_upper, 10):
        #plot_save_path = "../../plot/co_train_detail_plots/threshold="+str(step_threshold)+"/binary_sample=140k/"+select_emb+"/"
        threshold_change.append(step_threshold)
        # collect statistic to output
        name_group, total_sample_size, train_sample_size, test_sample_size= ([] for i in range(4))
        unlabeled_count, co_train_self_labeled = ([] for i in range(2))

        all_LR_f1,all_SVM_f1, all_co_LR_f1 = ([] for i in range(3))
        all_per_fold_f1_score_variance = []

        total_selected_group = 0

        # ------- different name group in all name group --------------------#
        for file in listfiles:
            # group name
            temp = file.split("_")
            name = temp[1]+"_"+temp[-1]
            print("For name: ",name)
            # read labeled pid and aid from file
            data = com_func.read_pid_aid(fileDir+file)
            labeled_mask = data["authorID"] != "-1"
            labeled_data = data[labeled_mask]
            print(labeled_data.shape)
            # ---------------- collect all labeled sample -------------------- #
            # ---------------- if use all samples as negative --------------- #
            all_labeled_samples = labeled_data["paperID"].tolist()
            authorCounter = com_func.select_productive_groups(labeled_data, threshold_select_name_group)
            # if only have one class or no class pass the threshold, not applicable
            if(len(authorCounter)==0) or (len(authorCounter)==1):
                print(name," pass")
            else:
                # --------------for each name group---------------- #
                if apply_threshold_to_name_group_samples == True:
                    # ---------- only use sample pass threshold ------- #
                    #-------- only select authors in name group are very productive (more than threshold)---------#
                    labeled_data, author_list, _= com_func.only_select_productive_authors(labeled_data, step_threshold)
                    # ----------------- if use filtered samples as negative  --------- #
                    filtered_all_labeled_samples = labeled_data["paperID"].tolist()
                else:
                    # ----------- use all sample in name group --------- #
                    author_list = com_func.productive_authors_list(labeled_data, step_threshold)
                    print(name, " name group sample size: ",labeled_data.shape)
                # -------------- extract all samples for name group -------------- #
                # for each name group
                # read in labeled data
                labeled_viewone_text = com_func.extract_sorted_embedding(viewone_text_emb, labeled_data["paperID"])
                print(labeled_viewone_text.shape)
                labeled_viewtwo_citation = com_func.extract_sorted_embedding(viewtwo_citation_embedding, labeled_data["paperID"])
                print(labeled_viewtwo_citation.shape)
                print("Labeled: ",len(labeled_viewone_text), " : ", len(labeled_viewtwo_citation))
                # ---------------- shuffle the data ----------------- #
                labeled_data = labeled_data.sample(frac=1).reset_index(drop=True)
                # ------------------ alignment ---------------------- #
                labeled_viewone_text = pd.merge(labeled_data, labeled_viewone_text, left_on="paperID", right_on = [0], how = "left")
                labeled_viewtwo_citation = pd.merge(labeled_data, labeled_viewtwo_citation, left_on="paperID", right_on = [0], how = "left")
                labeled_viewtwo_citation.fillna(0, inplace=True)
                unique_labels = labeled_viewone_text.authorID.unique()
                map_dict = {}
                for idx, unique_label in enumerate(unique_labels):
                    map_dict[unique_label] = name+"_"+str(idx)
                true_label = labeled_viewone_text["authorID"].replace(map_dict)
                
                '''
                only work on binary case, ignored multi-class case
                We need to check whether the name group only contain binary case or not
                '''
                if len(author_list) == 2:
                    total_selected_group+= 1
                    name_group.append(name)
                    print(name + " is binary case")
                    viewone_text_final = labeled_viewone_text.drop(["paperID", "authorID", 0], axis=1)
                    viewtwo_citation_final = labeled_viewtwo_citation.drop(["paperID", "authorID", 0], axis=1)
                    # apply co-training
                    co_logistic_clf = Co_training_clf(clf1=LogisticRegression(solver= "liblinear"),p=1,n=1, k=30)
                    LR_f1, SVM_f1, co_lr_f1, name_per_fold_status= k_fold_cv_co_train_binary(viewone_text_final, viewtwo_citation_final,
                                                                                             true_label,init_labeled_size, co_logistic_clf,10)
                    total_sample_size.append(len(true_label))
                    train_sample_size.append(name_per_fold_status[0]["train_size"])
                    test_sample_size.append(name_per_fold_status[0]["test_size"])
                    unlabeled_count.append(name_per_fold_status[0]["unlabeled size"])
                    co_train_self_labeled.append(name_per_fold_status[0]["total_self_labeled_train"])
                    all_LR_f1.append(LR_f1)
                    all_SVM_f1.append(SVM_f1)
                    all_co_LR_f1.append(co_lr_f1)
                else:
                    print(name+ " is multi-class case, ignored")
                    
#         # write evaluation result to excel
#         output = pd.DataFrame({'Name':name_group, "Total sample size":total_sample_size, "train size":train_sample_size,
#                                "test size":test_sample_size, "unlabeled sample size": unlabeled_count, 
#                                "total self labeled sample":co_train_self_labeled,
#                                "LR F1": all_LR_f1, "SVM F1": all_SVM_f1, "co_logisticRegression F1": all_co_LR_f1})
#         savePath = "../../result/"+Dataset+"/co_train_binary_140k/"
#         filename = "(Labeled_size="+str(init_labeled_size)+") V1TextEmb="+select_emb+"_V2CitationEmb="+pp_citation+"_threshold="+str(step_threshold)+"_namegroupcount="+str(total_selected_group)+".csv"
#         com_func.write_csv_df(savePath, filename, output)
#         print("Done")
        
        threshold_change_all_co_lr_f1s.append(all_co_LR_f1)
        
    co_lr_diff_embedding_result.append(threshold_change_all_co_lr_f1s)

Load text embedding:  pv_dbow
Total text vector records: 135796
Vector dimension:  100
Load citation embedding:  n2v
Total citation vector records: 124922
Vector dimension:  101
For name:  j_read
(136, 2)
j_read  pass
For name:  f_esteves
(34, 2)
f_esteves  pass
For name:  c_miller
(252, 2)
c_miller  pass
For name:  r_jha
(11, 2)
r_jha  pass
For name:  a_lowe
(102, 2)
a_lowe  pass
For name:  a_vega
(20, 2)
a_vega  pass
For name:  k_smith
(338, 2)
k_smith  pass
For name:  j_gordon
(19, 2)
j_gordon  pass
For name:  s_liao
(104, 2)
s_liao  pass
For name:  j_qian
(17, 2)
j_qian  pass
For name:  s_bernardi
(91, 2)
s_bernardi  pass
For name:  t_hill
(15, 2)
t_hill  pass
For name:  s_schindler
(51, 2)
s_schindler  pass
For name:  j_williams
(625, 2)
j_williams  pass
For name:  s_jacobson
(28, 2)
s_jacobson  pass
For name:  e_andrade
(17, 2)
e_andrade  pass
For name:  t_santos
(45, 2)
t_santos  pass
For name:  k_kim
(1111, 2)
Total sample size before apply threshold:  1111
Counter({'0000-0002-

Total missing sample:  0
(252, 101)
Total missing sample:  6
(252, 101)
Labeled:  252  :  252
(252, 103)
(252, 103)
p_robinson is binary case
[('p_robinson_0', 0.4722222222222222), ('p_robinson_1', 0.5277777777777778)]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'p_robinson_0', -1, 'p_robinson_0', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'p_robinson_1', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'p_robinson_0', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'p_robinson_0', -1, -1, -1, -1, 'p_robinson_1', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'p_robinson_1', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 

N:  69  :  [0.0183 0.9817]
p_robinson_0  (u' idx):  {74, 84}
p_robinson_0  (U idx):  [96, 3]
p_robinson_1  (u' idx):  {68, 69}
p_robinson_1  (U idx):  [6, 221]
[([84], [74]), ([68], [69])]
[96, 3, 6, 221]
Total Labeled number:  120  Still unlabeled number:  85
30
30
Self labeled sample index:  defaultdict(<class 'list'>, {'p_robinson_0': [[9, 68], [46, 105], [85, 37], [165, 168], [55, 53], [137], [112, 56], [198, 130], [45, 133], [206, 104], [52, 155], [107, 58], [212], [146, 26], [50], [195, 124], [182], [11, 1], [186], [111, 211], [90, 147], [119, 157], [66, 136], [67, 73], [209, 30], [0, 83], [41], [100, 47], [202, 194], [96, 3]], 'p_robinson_1': [[190, 94], [43, 163], [103, 23], [185, 82], [173, 28], [207, 59], [166, 89], [88, 7], [84, 154], [218, 175], [176, 77], [169, 108], [109, 204], [162, 135], [126, 150], [134], [203, 38], [31, 110], [181], [223, 95], [138, 125], [36, 79], [93], [49], [98, 86], [101, 193], [116, 219], [42, 167], [57, 91], [6, 221]]})
y1 disagree on 0  Proba: 

P:  75  :  [0.9905 0.0095]
N:  20  :  [0.0358 0.9642]
P:  75  :  [0.9953 0.0047]
N:  50  :  [0.0363 0.9637]
p_robinson_0  (u' idx):  {75}
p_robinson_0  (U idx):  [130]
p_robinson_1  (u' idx):  {50, 20}
p_robinson_1  (U idx):  [95, 144]
[([75], [75]), ([20], [50])]
[130, 95, 144]
P:  55  :  [0.9629 0.0371]
N:  76  :  [0.0069 0.9931]
P:  72  :  [0.9754 0.0246]
N:  76  :  [0.0137 0.9863]
p_robinson_0  (u' idx):  {72, 55}
p_robinson_0  (U idx):  [129, 114]
p_robinson_1  (u' idx):  {76}
p_robinson_1  (U idx):  [49]
[([55], [72]), ([76], [76])]
[129, 114, 49]
P:  79  :  [0.9893 0.0107]
N:  26  :  [0.0318 0.9682]
P:  80  :  [0.9784 0.0216]
N:  28  :  [0.0315 0.9685]
p_robinson_0  (u' idx):  {80, 79}
p_robinson_0  (U idx):  [198, 133]
p_robinson_1  (u' idx):  {26, 28}
p_robinson_1  (U idx):  [57, 33]
[([79], [80]), ([26], [28])]
[198, 133, 57, 33]
P:  46  :  [0.966 0.034]
N:  60  :  [0.0311 0.9689]
P:  5  :  [0.9709 0.0291]
N:  41  :  [0.0329 0.9671]
p_robinson_0  (u' idx):  {5, 46}
p_robinson

P:  10  :  [0.9644 0.0356]
N:  45  :  [0.0487 0.9513]
P:  20  :  [0.9647 0.0353]
N:  34  :  [0.0366 0.9634]
p_robinson_0  (u' idx):  {10, 20}
p_robinson_0  (U idx):  [147, 8]
p_robinson_1  (u' idx):  {34, 45}
p_robinson_1  (U idx):  [218, 224]
[([10], [20]), ([45], [34])]
[147, 8, 218, 224]
P:  6  :  [0.957 0.043]
N:  74  :  [0.0463 0.9537]
P:  6  :  [0.9652 0.0348]
N:  22  :  [0.0399 0.9601]
p_robinson_0  (u' idx):  {6}
p_robinson_0  (U idx):  [214]
p_robinson_1  (u' idx):  {74, 22}
p_robinson_1  (U idx):  [77, 162]
[([6], [6]), ([74], [22])]
[214, 77, 162]
P:  56  :  [0.9559 0.0441]
N:  75  :  [0.0314 0.9686]
P:  56  :  [0.9532 0.0468]
N:  57  :  [0.0489 0.9511]
p_robinson_0  (u' idx):  {56}
p_robinson_0  (U idx):  [53]
p_robinson_1  (u' idx):  {57, 75}
p_robinson_1  (U idx):  [50, 47]
[([56], [56]), ([75], [57])]
[53, 50, 47]
P:  11  :  [0.9433 0.0567]
N:  77  :  [0.0253 0.9747]
P:  13  :  [0.9455 0.0545]
N:  75  :  [0.0136 0.9864]
p_robinson_0  (u' idx):  {11, 13}
p_robinson_0  (U 

...

P:  47  :  [0.9644 0.0356]
N:  56  :  [0.0514 0.9486]
P:  54  :  [0.9741 0.0259]
N:  15  :  [0.171 0.829]
d_richardson_0  (u' idx):  {54, 47}
d_richardson_0  (U idx):  [262, 128]
d_richardson_1  (u' idx):  {56, 15}
d_richardson_1  (U idx):  [270, 277]
[([47], [54]), ([56], [15])]
[262, 128, 270, 277]
P:  69  :  [0.9667 0.0333]
N:  2  :  [0.0475 0.9525]
P:  37  :  [0.978 0.022]
N:  58  :  [0.2062 0.7938]
d_richardson_0  (u' idx):  {37, 69}
d_richardson_0  (U idx):  [357, 180]
d_richardson_1  (u' idx):  {2, 58}
d_richardson_1  (U idx):  [130, 189]
[([69], [37]), ([2], [58])]
[357, 180, 130, 189]
P:  2  :  [0.9681 0.0319]
N:  21  :  [0.0416 0.9584]
P:  73  :  [0.9909 0.0091]
N:  65  :  [0.1853 0.8147]
d_richardson_0  (u' idx):  {73, 2}
d_richardson_0  (U idx):  [79, 221]
d_richardson_1  (u' idx):  {65, 21}
d_richardson_1  (U idx):  [210, 224]
[([2], [73]), ([21], [65])]
[79, 221, 210, 224]
P:  11  :  [0.9719 0.0281]
N:  26  :  [0.0377 0.9623]
P:  27  :  [0.9819 0.0181]
N:  20  :  [0.1801 

                precision    recall  f1-score   support

d_richardson_0       0.97      1.00      0.99       231
d_richardson_1       1.00      0.96      0.98       167

     micro avg       0.98      0.98      0.98       398
     macro avg       0.99      0.98      0.98       398
  weighted avg       0.99      0.98      0.98       398

[231   0   6 161]
                precision    recall  f1-score   support

d_richardson_0       0.97      0.99      0.98       231
d_richardson_1       0.99      0.96      0.97       167

     micro avg       0.98      0.98      0.98       398
     macro avg       0.98      0.97      0.98       398
  weighted avg       0.98      0.98      0.98       398

[229   2   7 160]
For name:  j_moraes
(26, 2)
j_moraes  pass
For name:  e_moreno
(83, 2)
e_moreno  pass
For name:  r_little
(4, 2)
r_little  pass
For name:  t_kobayashi
(150, 2)
t_kobayashi  pass
For name:  a_lin
(46, 2)
a_lin  pass
For name:  a_miranda
(70, 2)
a_miranda  pass
For name:  h_vogel
(15, 2)

N:  73  :  [0.0991 0.9009]
P:  5  :  [0.9422 0.0578]
N:  12  :  [0.0757 0.9243]
y_wang_0  (u' idx):  {60, 5}
y_wang_0  (U idx):  [140, 145]
y_wang_1  (u' idx):  {73, 12}
y_wang_1  (U idx):  [40, 81]
[([60], [5]), ([73], [12])]
[140, 145, 40, 81]
P:  3  :  [0.8917 0.1083]
N:  52  :  [0.0827 0.9173]
P:  59  :  [0.9569 0.0431]
N:  10  :  [0.0568 0.9432]
y_wang_0  (u' idx):  {59, 3}
y_wang_0  (U idx):  [170, 157]
y_wang_1  (u' idx):  {10, 52}
y_wang_1  (U idx):  [205, 212]
[([3], [59]), ([52], [10])]
[170, 157, 205, 212]
P:  38  :  [0.9061 0.0939]
N:  71  :  [0.079 0.921]
P:  4  :  [0.9209 0.0791]
N:  73  :  [0.0367 0.9633]
y_wang_0  (u' idx):  {4, 38}
y_wang_0  (U idx):  [78, 84]
y_wang_1  (u' idx):  {73, 71}
y_wang_1  (U idx):  [114, 127]
[([38], [4]), ([71], [73])]
[78, 84, 114, 127]
P:  20  :  [0.9131 0.0869]
N:  65  :  [0.0606 0.9394]
P:  58  :  [0.9279 0.0721]
N:  74  :  [0.0256 0.9744]
y_wang_0  (u' idx):  {58, 20}
y_wang_0  (U idx):  [137, 46]
y_wang_1  (u' idx):  {65, 74}
y_wang_1

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'y_wang_0', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'y_wang_1', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'y_wang_1', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'y_wang_1', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'y_wang_0', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'y_wang_0', 'y_wang_1', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 'y_wang_0', -1, -1, 'y_wang_1', -1, -1, 'y_wang_0', -1, -1, -1]
P value:  1  N value:  1
Initial L size:  10
Initial U size:  2

KeyboardInterrupt: 

In [18]:
from statistics import mean 

print(threshold_change_all_co_lr_f1s)
print(co_lr_diff_embedding_result)

[[0.9761167624944714, 0.9923460612315101, 0.9794320560192242, 0.9579713216076853, 0.966326695807026, 0.9819839830205209, 0.9922126745435016, 0.9424861608596549, 0.9944401544401544, 0.9732348111658456, 0.939352110194132, 0.9924358974358973, 0.976890756302521, 0.9293797303430003, 0.9883887801696021]]
[[[0.9761167624944714, 0.9923460612315101, 0.9794320560192242, 0.9579713216076853, 0.966326695807026, 0.9819839830205209, 0.9922126745435016, 0.9424861608596549, 0.9944401544401544, 0.9732348111658456, 0.939352110194132, 0.9924358974358973, 0.976890756302521, 0.9293797303430003, 0.9883887801696021]]]


In [ ]:
# %whos
del viewtwo_citation_embedding
del viewone_text_emb